# Update

Es ist Echt schwer die Vergleiche zwischen der Repo und dem Guide zu ziehen. Ich probiere jetzt nochmal Das gleiche wie im guide zu erreichen mit dem Vergleich aus dem Guide zur language translation -> siehe anderes Notebook

# First own Implementation Test for Transformer Generally (not Transformer-decoder only)

Ich Richte mich nach Folgenden Guides:

- Eher Beginner Guide: https://towardsdatascience.com/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
- Pytorch official word_language_model example: https://github.com/pytorch/examples/blob/86591de88a25944c2d1d72506f04af79b4301cc3/word_language_model
- (Pytorch official Language Translation: https://pytorch.org/tutorials/beginner/translation_transformer.html) the other two are the main sources


Ich benötige Folgende Dinge, um ein Modell zu trainieren:

## Daten

## Modell

Ich glaube so dinge wie Das positional embedding und da Vektorembedding muss selbst implementiert werden. Es gibt kein Vorgefertigtes pytorch modul

- Die Outputs müssen für den Decoder eins nach rechts geschoben werden (hier sind dann auch die Special Tokens wichtig
- Die Die Eingaben in die Attention-Blöcke müssen von Vektoren zu Embedding vektoren umgewandelt werden
- Dann das ganze Transformer zeug
- Dann Linear Schicht
- Dann Softmax schicht für die Wahrscheinlichkeiten

## Imports

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import math
import numpy as np

import random

# Wir Prauchen das Positional embedding, das den Tokens informationen über ihre Reihenfolge dazugibt

This part is basically the same in the Guide and the official github repo. Repo has very slight differences in Code, Guide has comments:

I think there is no implementation in Pytorch for thsi

- Guide: https://towardsdatascience.com/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
- pytorch example: https://github.com/pytorch/examples/blob/main/word_language_model/model.py

In [10]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Info
        self.dropout = nn.Dropout(p=dropout)
        
        # Encoding - From formula -> This is basically applying the formula for Positional encoding (The one with Sinus and Cosinus)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # Baically a positions list 0, 1, 2, 3, 4, 5, ...
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # 1000^(2i/dim_model)
        
        # # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        
        #  # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pe[:, 1::2] = torch.cos(position * div_term)
        
         # Saving buffer (same as parameter without gradients needed)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """
        
        # Residual connection + pos encoding
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


# Jetzt die Basic Transformer-Struktur Definieren

Here are Big differnces between the Guide and the example repo im using the Code from the Repo and trying to understand and comment what it does with the help of the guide

Ich habe keine Ahnung, ob hier das masking und Padding gemacht wird oder nicht

Zumindest das masking muss selber gemacht werden glaube ich 
-> Wir müssen Das masking selber definieren

In [11]:
class TransformerModel(nn.Transformer):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    # ntoken -> number of tokens -> Wahrscheinlich wie viele Verschiedene Tokens es zur eingabe gibt.
    # z.B. wenn wir die Tokens 0 und 1 als daten haben und <EOS> (End of stream) und ein Padding token, dann haben wir 4 Tokens
    
    # d_model =  the number of expected features in the encoder/decoder inputs (default=512).
    # -> variable nimp steht wahrscheinlich für number_of_inputs
    # Wenn ich eine Sequenz von 8 Wörtern habe, dann muss mein d_model 8  sein. z.b. bei den Trainingsdaten 1, 0, 1, 0, 1, 0, 1, 0 → 1, 0, 1, 0, 1, 0, 1, 0
    
    # nhead (int) – the number of heads in the multiheadattention models (default=8).
    
    # dim_feedforward (int) – the dimension of the feedforward network model (default=2048).
    # -> nhid steht wahrscheinlich für n_hidden_layers (welche dimension das feed forward network hat)
    
    # num_encoder_layers (int) – the number of sub-encoder-layers in the encoder (default=6).
    # -> wie viele Encoder-blöcke es geben soll

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__(d_model=ninp, nhead=nhead, dim_feedforward=nhid, num_encoder_layers=nlayers)
        self.model_type = 'Transformer'
        self.src_mask = None
        
        ## Hier werden die Layers definiert glaube ich
        self.pos_encoder = PositionalEncoding(ninp, dropout) # This is probably the Positional encoding defined earlier

        self.input_emb = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):
        # Hier wird evtl die Maske für Masked attention generiert?
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.input_emb(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.encoder(src, mask=self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)
        
        

# Getting Data

Ich Versuche die Daten aus dem Guide zu nehmen, um zu schauen ob es mit dem Transformer aus dem Example aus der Repo funktioniert

In [12]:


def generate_random_data(n):
    SOS_token = np.array([2])
    EOS_token = np.array([3])
    length = 8

    data = []

    # 1,1,1,1,1,1 -> 1,1,1,1,1
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.ones(length), EOS_token))
        y = np.concatenate((SOS_token, np.ones(length), EOS_token))
        data.append([X, y])

    # 0,0,0,0 -> 0,0,0,0
    for i in range(n // 3):
        X = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        y = np.concatenate((SOS_token, np.zeros(length), EOS_token))
        data.append([X, y])

    # 1,0,1,0 -> 1,0,1,0,1
    for i in range(n // 3):
        X = np.zeros(length)
        start = random.randint(0, 1)

        X[start::2] = 1

        y = np.zeros(length)
        if X[-1] == 0:
            y[::2] = 1
        else:
            y[1::2] = 1

        X = np.concatenate((SOS_token, X, EOS_token))
        y = np.concatenate((SOS_token, y, EOS_token))

        data.append([X, y])

    np.random.shuffle(data)

    return data


def batchify_data(data, batch_size=16, padding=False, padding_token=-1):
    batches = []
    for idx in range(0, len(data), batch_size):
        # We make sure we dont get the last bit if its not batch_size size
        if idx + batch_size < len(data):
            # Here you would need to get the max length of the batch,
            # and normalize the length with the PAD token.
            if padding:
                max_batch_length = 0

                # Get longest sentence in batch
                for seq in data[idx : idx + batch_size]:
                    if len(seq) > max_batch_length:
                        max_batch_length = len(seq)

                # Append X padding tokens until it reaches the max length
                for seq_idx in range(batch_size):
                    remaining_length = max_batch_length - len(data[idx + seq_idx])
                    data[idx + seq_idx] += [padding_token] * remaining_length

            batches.append(np.array(data[idx : idx + batch_size]).astype(np.int64))

    print(f"{len(batches)} batches of size {batch_size}")

    return batches


train_data = generate_random_data(9000)
val_data = generate_random_data(3000)

train_dataloader = batchify_data(train_data)
val_dataloader = batchify_data(val_data)

# Jetzt das Modell Erstellen und trainieren

gpu nutzen wenn möglich

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(device)

Die Parameter des Modells festlegen (Muss sehr wahrscheinlich an daten angepasst werden)

hier muss model importiert werden? -> nein ist in der Repo nur in verschiedene Dateien aufgeteilt

Im repo beispiel werden viele der Parameter als Argumente beim starten des Scripts übergeben. Deshalb die args.something_here im Code in der Repo

Die Ganzen Hyperparameter für das Modell müssen noch evtl noch angepasst werden. Dafür evtl. Guide, Repo und offizielle Dokumentation vergleichen. Genau nachprüfen, was die Hyperparameter alle bedeuten.

### Siehe Modelldefinition Kommentare für eventuelle Erklärung

`args.emsize` ist in der Repo die size of word embeddings. Ist das das gleiche wie dim_model im Guide ?

In [14]:
num_tokens = 4 # in der Repo als len(corpus.dictionary) -> so viele Wörter gibt es
dim_model = 8
num_heads=2 
num_encoder_layers=3
num_decoder_layers=3
dropout_p=0.1

num_hid = 128

model = TransformerModel(num_tokens, dim_model, num_heads, nhid=num_hid, nlayers=num_encoder_layers, dropout=dropout_p).to(device)

## Jetzt ist das Modell erstellt und jetzt müssen Kostenfunktion und ein Optimizer festgelegt werden

wird beides im Guide direkt nach erstellen des Modells festgelegt. 

ist das hier die Kostenfunktion in der Repo? `criterion = nn.NLLLoss()`

wo ist der optimizer in der Repo?

ich nehm mal das aus dem Guide

In [15]:
opt = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

## Jetzt müssen Trainings und validation loop erstellt werden.

### Ich definiere zuerst den Trainings loop (ich halte mich an den aus dem Guide) -> Das repo model hat kein get_tgt_mask 

Wie wird das masking gemacht? 

ich probiere jetzt mal eine mischung aus dem guide und der Repo

In [16]:
def train_loop(model, opt, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        X, y = batch[:, 0], batch[:, 1]
        X, y = torch.tensor(X).to(device), torch.tensor(y).to(device)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]
        
        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)

        # Standard training except we pass in y_input and tgt_mask
        pred = model(X, y_input, tgt_mask)

        # Permute pred to have batch size first again
        pred = pred.permute(1, 2, 0)      
        loss = loss_fn(pred, y_expected)

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

### Dann wird der Validation-loop definiert

In [17]:
def validation_loop(model, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch[:, 0], batch[:, 1]
            X, y = torch.tensor(X, dtype=torch.long, device=device), torch.tensor(y, dtype=torch.long, device=device)

            # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
            y_input = y[:,:-1]
            y_expected = y[:,1:]
            
            # Get mask to mask out the next words
            sequence_length = y_input.size(1)
            tgt_mask = model.get_tgt_mask(sequence_length).to(device)

            # Standard training except we pass in y_input and src_mask
            pred = model(X, y_input, tgt_mask)

            # Permute pred to have batch size first again
            pred = pred.permute(1, 2, 0)      
            loss = loss_fn(pred, y_expected)
            total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

## Jetzt das Modell trainieren

In [18]:
def fit(model, opt, loss_fn, train_dataloader, val_dataloader, epochs):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    # Used for plotting later on
    train_loss_list, validation_loss_list = [], []
    
    print("Training and validating model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        
        train_loss = train_loop(model, opt, loss_fn, train_dataloader)
        train_loss_list += [train_loss]
        
        validation_loss = validation_loop(model, loss_fn, val_dataloader)
        validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
        print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list, validation_loss_list
    
train_loss_list, validation_loss_list = fit(model, opt, loss_fn, train_dataloader, val_dataloader, 10)